# Preamble

In [ ]:
!pip3 install git+https://github.com/aims50toolbox/aitoolbox.git

# Setup

In [ ]:
# import local aitoolbox support library packages
from aitoolbox.context import Context
from aitoolbox.datatypes.image import Image

In [ ]:
#!skip

# define test inputs
Context.get().get_sources().set("system_prompt","Please answer like a poet!")
Context.get().get_sources().set("user_prompt","What is the ultimate answer to life, the universe and everything?")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [ ]:
# token for some pretty basic authorization
TOKEN="hZmAMwYmrG7Xvxsg8KNmLxZ9TWWJvA"

Here, please provide **Hugging Face token**. This is required to download licensed models (see https://huggingface.co/docs/hub/en/security-tokens)

In [ ]:
# HuggingFace token
HF_TOKEN = '<HF-TOKEN>'

Please change the `model_id` to execute different models from Hugging Face. See e.g. https://huggingface.co/LLMs.

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
    token = HF_TOKEN
)

# Service

In [ ]:
import aitoolbox.errors as aierr
if Context.get().get_sources().get("token") != TOKEN:
    raise aierr.ServiceError("Token authorization failed")

In [ ]:
messages = [
    {"role": "system", "content": Context.get().get_sources().get("system_prompt")},
    {"role": "user", "content": Context.get().get_sources().get("user_prompt")},
]

In [ ]:
prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=4096,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
out_text = outputs[0]["generated_text"][len(prompt):]

In [ ]:
#!skip
print(out_text)

In [ ]:
Context.get().get_destinations().set("resp",out_text)